<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li></ul></div>

# Introduction

This project is focused on building a scraper bot that regularly scans all listings on Immobilienscout24 and applies to those that fit the predefined criteria.

Acceptance criteria:
- The bot can open a webbrowers, login with an account and scan listings
- The bot can identify listings of interest and apply to them using an editable message template
- A history of all applied to apartments should be viewable
- Does not to send applications to apartments that were already applied to
- Listing scan frequency should be adjustable


Bonus:
- Email alert or sms is sent asking user if the apartment should be applied to
- User can answer "yes" or "no"

In [1]:
# imports

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random



In [2]:
def sleep_randomly():
    amount = random.randint(2,5)

In [6]:
# driver

from time import sleep

import undetected_chromedriver as uc

driver = uc.Chrome()

url = "https://sso.immobilienscout24.de/sso/login?appName=is24main&source=meinkontodropdown-login&sso_return=https://www.immobilienscout24.de/sso/login.go?source%3Dmeinkontodropdown-login%26returnUrl%3D/geschlossenerbereich/start.html?source%253Dmeinkontodropdown-login"
driver.get(url)

sleep(6)


# WORKS!!
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', element)
    return shadow_root

try:
    shadow_outer = expand_shadow_element(driver.find_element_by_css_selector("div#usercentrics-root"))
    accept_cookie = shadow_outer.find_element(by=By.CSS_SELECTOR, value="button[data-testid='uc-accept-all-button']")
    accept_cookie.click()
except:
    pass

sleep_randomly()

# sign in - user
user_name = driver.find_element(by=By.ID, value="username")
user_name.send_keys("moe.joe4411@gmail.com")
sleep_randomly()

user_name.send_keys(Keys.RETURN)

# sign in - password
sleep_randomly()
password = driver.find_element(by=By.ID, value="password")
password.send_keys("My_apartment_1")
password.send_keys(Keys.RETURN)

driver.get("https://www.immobilienscout24.de/Suche/shape/wohnung-mieten?shape=b3ZwX0l9aX1vQX58QW9wQWR3Qmt0Snhbb2BHaGBBc3pJc0FjfkVvekF9aUBlZkRpYUBlb0Z_Y0N9YEB2X0N5fkJoaEN8YEBsZEV8bUBuakNgakBueUN4YEJ4eUU.&haspromotion=false&numberofrooms=3.0-&price=-1350.0&exclusioncriteria=swapflat&pricetype=rentpermonth&enteredFrom=saved_search")
sleep_randomly()


# get all listing Id's

def get_new_listings():
    result_list = driver.find_element(by=By.ID, value='listings')


    listings_seen = ["133536717"]

    all_listings = result_list.find_elements_by_tag_name("li")    
    new_listings = [listing for listing in all_listings if listing.get_attribute("data-id") not in listings_seen]

    
    return new_listings

# check if listing was already seen/ applied to

new_listings = get_new_listings()

listing_ids = [li.get_attribute("data-id") for li in new_listings if li.get_attribute("data-id") !=None]

for li_id in listing_ids:
    
    print(li_id)

    # go to expose
    expose_url = f'https://www.immobilienscout24.de/expose/{li_id}'
    
    
    

    driver.get(expose_url)
    # click on contact
    
    price = driver.find_element(by=By.XPATH, value='//*[@id="is24-content"]/div[2]/div[1]/div[2]/div[1]/div/div[1]').text
    location = driver.find_element(by=By.CLASS_NAME, value="address-block").text
    contact_name = driver.find_element(by=By.XPATH, value='//*[@id="is24-expose-contact-box"]/div/div/div[2]/div[1]/div/div[1]').text
    print(price, location, contact_name)
    
    
    contact = driver.find_element(by=By.ID, value="contact-realtor-desk").click()
    sleep(4)
    
    
    # fill in message
    
    def get_message(contact_name=None, location=None):
        
        title = None         
        if "herr" in contact_name.lower():
            last_name = contact_name#.split()[-1]
            greetings = f"Sehr geehrter Herr {last_name},"
                        
            
        elif "frau" in contact_name.lower():
            last_name = contact_name.split()[-1]
            greetings = f"Sehr geehrte Frau {last_name},"
            
        else:
            greetings = f"Sehr geehrte Damen und Herren,"
            
        
        location = location.split(",")[0].split()[0]
        
        
        message_template = f"""{greetings}
        
wir sind ein Paar aus Berlin und suchen nun eine größere Wohnung, um unser Leben als Ehepaar zu beginnen. Wir sind beide 28 Jahre alt, berufstätig und haben ein Gesamteinkommen von 5000 EUR netto pro Monat. Ihre schöne 3-Zimmer-Wohnung in {location} ist uns sehr aufgefallen und wir würden uns über einen Besichtigungstermin sehr freuen. Vielen Dank im Voraus.
        
Mit freundlichen Grüßen
Moe und Allegra
        """
        
        return message_template
    
    

    
    message_text = driver.find_element(by=By.ID, value="contactForm-Message")
    
    message_text.send_keys(get_message(contact_name=contact_name, location=location))
    sleep_randomly()
    sleep(10)
    






/var/folders/vp/w0q6bwv17zg47jg_mhc4nsdh0000gp/T/ipykernel_62568/1144861409.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  shadow_outer = expand_shadow_element(driver.find_element_by_css_selector("div#usercentrics-root"))
/var/folders/vp/w0q6bwv17zg47jg_mhc4nsdh0000gp/T/ipykernel_62568/1144861409.py:54: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  all_listings = result_list.find_elements_by_tag_name("li")


112355606
1.022 € Moabit, 10551 Berlin
Die vollständige Adresse der Immobilie erhalten Sie vom Anbieter. Frau Dipl.Kff. Suzana Huseini SH IMMMOBILIEN in Koop. m VBRE
104750126
1.129 € Wollankstr. 132,
Pankow (Ortsteil), 13187 Berlin Wohnen - Düsseldorf
100239203
1.200 € Birkenstr. xx,
Moabit, 10559 Berlin Herr Shababzadeh
135374446
856,16 € Mainzer Str. 15,
Friedrichshain, 10247 Berlin Frau Böhringer
135163012
1.150 € Charlottenburg, 10589 Berlin
Die vollständige Adresse der Immobilie erhalten Sie vom Anbieter. Herr Dennis Delikat
135323523


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   8709099323f1e378_chromedriver       0x0000000100939ef9 8709099323f1e378_chromedriver + 4480761
1   8709099323f1e378_chromedriver       0x00000001008c55d3 8709099323f1e378_chromedriver + 4003283
2   8709099323f1e378_chromedriver       0x0000000100558338 8709099323f1e378_chromedriver + 410424
3   8709099323f1e378_chromedriver       0x000000010054880f 8709099323f1e378_chromedriver + 346127
4   8709099323f1e378_chromedriver       0x0000000100549cb2 8709099323f1e378_chromedriver + 351410
5   8709099323f1e378_chromedriver       0x00000001005424dc 8709099323f1e378_chromedriver + 320732
6   8709099323f1e378_chromedriver       0x0000000100559752 8709099323f1e378_chromedriver + 415570
7   8709099323f1e378_chromedriver       0x00000001005bdbab 8709099323f1e378_chromedriver + 826283
8   8709099323f1e378_chromedriver       0x00000001005ab953 8709099323f1e378_chromedriver